In [1]:
print('oi')
#! pip install tensorflow
#! pip install tensorflow_datasets
#! pip install tensorflow.io



        

oi


In [2]:
#!pip install --ignore-installed tensorflow

In [3]:
import numpy as np
import pandas as pd   
import os
from pathlib import Path
import glob
import json
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import nltk
import cv2
import matplotlib.pyplot as plt
import random
from keras.preprocessing import text
print(tf.__version__)
print('fim')
from keras.backend import manual_variable_initialization
manual_variable_initialization(True)

2.6.0-rc0
fim


In [4]:
#Check GPU is available for training or not Or whether the tensorflow version can utilize gpu 
physical_devices = tf.config.list_physical_devices('GPU') 
print("Number of GPUs :", len(physical_devices)) 
print("Tensorflow GPU :",tf.test.is_built_with_cuda())
if len(physical_devices)>0:
    device="/GPU:0"
else:
    device="/CPU:0"

    lenghtData = 10 
percentageData = 0.01
posTrainList=[]
posValList=[]
BATCH_SIZE=1
IMG_SIZE=(200,200)
QtdEpocasGravarCHKP = 5
Epochs = 10

Number of GPUs : 0
Tensorflow GPU : True


In [5]:
trainList=[]

with open('/home/jupyter/input/clevr-dataset/CLEVR_v1.0/questions/CLEVR_train_questions.json') as f:
    data = json.load(f)
    lenghtDataTrain = int(len(data['questions']) * (percentageData/100))

    for K in range(lenghtDataTrain):
        i = data['questions'][K]
       
        temp=[]
        for path in glob.glob('/home/jupyter/input/clevr-dataset/CLEVR_v1.0/images/train/'+i['image_filename']): 
            temp.append(path)
           
        temp.append(i['question'])
        temp.append(i['answer'])
        trainList.append(temp)
        posTrainList.append(K)
f.close()
labels=['Path','Question','Answer']
train_dataframe = pd.DataFrame.from_records(trainList, columns=labels)#training Dataframe 
del(data)
del(trainList)
print('fim')


fim


In [6]:
valList=[]
with open('/home/jupyter/input/clevr-dataset/CLEVR_v1.0/questions/CLEVR_val_questions.json') as f:
    data = json.load(f)
    lenghtDataTrain = int(len(data['questions']) * (percentageData/100))
    for K in range(lenghtDataTrain):
    
       
        i = data['questions'][K]
        
        temp=[]
        for path in glob.glob('/home/jupyter/input/clevr-dataset/CLEVR_v1.0/images/val/'+i['image_filename']): 
            temp.append(path)
        temp.append(i['question'])
        temp.append(i['answer'])
        valList.append(temp)
        posValList.append(K)
f.close() 

val_dataframe = pd.DataFrame.from_records(valList, columns=labels)#validation Dataframe
del(data)
del(valList)
val_dataframe.head()
print('passou carregou')

passou carregou


In [7]:
vocab_set=set()#set object used to store the vocabulary

tokenizer = tfds.features.text.Tokenizer()

for i in val_dataframe['Question']:
    vocab_set.update(tokenizer.tokenize(i))
for i in train_dataframe['Question']:
    vocab_set.update(tokenizer.tokenize(i))
for i in val_dataframe['Answer']:
    vocab_set.update(tokenizer.tokenize(i))
for i in train_dataframe['Answer']:
    vocab_set.update(tokenizer.tokenize(i))
    
vocab_set.update('12aaaa')
vocab_set.update('1234sssa')

#
#Creating an Encoder and a Function to preprocess the text data during the training and inference    
    
encoder=tfds.features.text.TokenTextEncoder(vocab_set)
index=2
print("Testing the Encoder with sample questions - \n ")
example_text=encoder.encode(train_dataframe['Question'][index])
print("Original Text = "+train_dataframe['Question'][index])
print("After Encoding = "+str(example_text))


Testing the Encoder with sample questions - 
 
Original Text = Is the color of the large sphere the same as the large matte cube?
After Encoding = [28, 18, 83, 37, 18, 72, 81, 18, 38, 78, 18, 72, 42, 69]


In [21]:
json_file = open('teste2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model2 = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
model2.load_weights("testenovo3")
print(tf.__version__)
model2.summary()

AttributeError: module 'keras.api._v2.keras.models' has no attribute 'load'

In [24]:
model2 = tf.keras.models.load_model("testenovo3")

In [27]:
contadorCerto =0
contadorErro =0 
#model.load_weights('/home/jupyter/imported/Percentage_0.01/weights-010.ckpt')
#print (tf.keras.models.load_model("my_model").get_weights())
model2 = tf.keras.models.load_model("testenovo3")
#model2.set_weights(tf.keras.models.load_model("my_model").get_weights())


for contador in range(69) :
   
    im=cv2.imread(train_dataframe.iloc[contador]['Path'])
    im=cv2.resize(im,(200,200))
    q=train_dataframe.iloc[contador]['Question'] 
    q=encoder.encode(q)
    paddings = [[0, 50-tf.shape(q)[0]]]
    q=tf.pad(q, paddings, 'CONSTANT', constant_values=0)
    q=np.array(q)
    im.resize(1,200,200,3)
    q.resize(1,50)
    ans=model2.predict([im,q]) 
    decodAns = encoder.decode([np.argmax(ans)])
    if train_dataframe.iloc[contador]['Answer'] != decodAns :
        contadorErro = contadorErro +1
    else:     
        contadorCerto = contadorCerto +1
      
print(contadorCerto)
print(contadorErro)
    

0
69


In [6]:
# load json and create model
json_file = open('teste2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model2 = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
model2.load_weights("teste2_weights.h5")
print("Loaded model from disk")



Loaded model from disk


In [7]:
model2.summary()
model2.losses

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 256)      23808       text_input[0][0]                 
__________________________________________________________________________________________________
image_input (InputLayer)        [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 50, 512)      1050624     embedding[0][0]                  
______________________________________________________________________________________________

[]

In [9]:
model2.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

contadorCerto =0
contadorErro =0 
#model.load_weights('/home/jupyter/imported/Percentage_0.01/weights-010.ckpt')
#print (tf.keras.models.load_model("my_model").get_weights())
#model2 = tf.keras.models.load_model("my_model")
#model2.set_weights(tf.keras.models.load_model("my_model").get_weights())


for contador in range(69) :
   
    im=cv2.imread(train_dataframe.iloc[contador]['Path'])
    im=cv2.resize(im,(200,200))
    q=train_dataframe.iloc[contador]['Question'] 
    q=encoder.encode(q)
    paddings = [[0, 50-tf.shape(q)[0]]]
    q=tf.pad(q, paddings, 'CONSTANT', constant_values=0)
    q=np.array(q)
    im.resize(1,200,200,3)
    q.resize(1,50)
    ans=model2.predict([im,q]) 
    decodAns = encoder.decode([np.argmax(ans)])
    if train_dataframe.iloc[contador]['Answer'] != decodAns :
        contadorErro = contadorErro +1
    else:     
        contadorCerto = contadorCerto +1
      
print(contadorCerto)
print(contadorErro)
    

0
69


In [40]:
tf.contrib.saved_model.save_keras_model(model2, './tmp_dir2')

AttributeError: module 'tensorflow' has no attribute 'contrib'

In [37]:
#Creating the CNN model for image processing


CNN_Input=tf.keras.layers.Input(shape=(200,200,3),name='image_input')

mobilenetv2=tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(200,200,3), alpha=1.0, include_top=False,
                                                      weights='imagenet', input_tensor=CNN_Input)

CNN_model=tf.keras.models.Sequential()
CNN_model.add(CNN_Input)
CNN_model.add(mobilenetv2)
CNN_model.add(tf.keras.layers.GlobalAveragePooling2D())



#Creating the RNN model for text processing
RNN_model=tf.keras.models.Sequential()

RNN_Input=tf.keras.layers.Input(shape=(50),name='text_input')
RNN_model.add(RNN_Input)
RNN_model.add(tf.keras.layers.Embedding (len(vocab_set)+1,256))
RNN_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,stateful=False,return_sequences=True,recurrent_initializer='glorot_uniform')))
RNN_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,stateful=False,return_sequences=True,recurrent_initializer='glorot_uniform')))
RNN_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512,stateful=False,return_sequences=False,recurrent_initializer='glorot_uniform')))


concat=tf.keras.layers.concatenate([CNN_model.output,RNN_model.output])
dense_out=tf.keras.layers.Dense(len(vocab_set)+1,activation='softmax',name='output')(concat)

model = tf.keras.Model(inputs=[CNN_Input,RNN_Input],
                    outputs=dense_out)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 256)      23808       text_input[0][0]                 
__________________________________________________________________________________________________
image_input (InputLayer)        [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 50, 512)      1050624     embedding[0][0]                  
______________________________________________________________________________________________

In [38]:
model2.load_weights("teste_weights.h5")
print("Loaded model from disk")

Loaded model from disk


In [39]:
contadorCerto =0
contadorErro =0 

for contador in range(69) :
   
    im=cv2.imread(train_dataframe.iloc[contador]['Path'])
    im=cv2.resize(im,(200,200))
    q=train_dataframe.iloc[contador]['Question'] 
    q=encoder.encode(q)
    paddings = [[0, 50-tf.shape(q)[0]]]
    q=tf.pad(q, paddings, 'CONSTANT', constant_values=0)
    q=np.array(q)
    im.resize(1,200,200,3)
    q.resize(1,50)
    ans=model.predict([im,q]) 
    decodAns = encoder.decode([np.argmax(ans)])
    if train_dataframe.iloc[contador]['Answer'] != decodAns :
        contadorErro = contadorErro +1
    else:     
        contadorCerto = contadorCerto +1
      
print(contadorCerto)
print(contadorErro)
    

0
69


In [ ]:
contadorCerto =0
contadorErro =0 
#model.load_weights('/home/jupyter/imported/Percentage_0.01/weights-010.ckpt')
#print (tf.keras.models.load_model("my_model").get_weights())
from tensorflow.keras.models import model_from_json

#loading model
model = model_from_json(open('/home/jupyter/imported/model_weights.h5').read())
model.load_weights('modelo.json')

#model2 = tf.keras.models.load_model("my_h5_model.h5")

for contador in range(69) :
   
    im=cv2.imread(train_dataframe.iloc[contador]['Path'])
    im=cv2.resize(im,(200,200))
    q=train_dataframe.iloc[contador]['Question'] 
    q=encoder.encode(q)
    paddings = [[0, 50-tf.shape(q)[0]]]
    q=tf.pad(q, paddings, 'CONSTANT', constant_values=0)
    q=np.array(q)
    im.resize(1,200,200,3)
    q.resize(1,50)
    ans=model2.predict([im,q]) 
    decodAns = encoder.decode([np.argmax(ans)])
    if train_dataframe.iloc[contador]['Answer'] != decodAns :
        contadorErro = contadorErro +1
    else:     
        contadorCerto = contadorCerto +1
      
print(contadorCerto)
print(contadorErro)
    